#  GPT-2 Generation and Fine-Tuning

This notebook explores GPT-2 (Generative Pretrained Transformer-2) from OpenAI. Read more about it [here](https://openai.com/blog/better-language-models/).

Activities include:

0. Setup
1. Generate samples from pre-trained gpt-2 model
2. Fine-tune gpt-2 on text of your choosing. 

Adapted by Robert Twomey (rtwomey@unl.edu) for Machine Learning for the Arts SP22 from this [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](http://minimaxir.com). See his repo [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple).

# 0. Setup

Run once to install the library

In [1]:
%pip install -q gpt-2-simple

Note: you may need to restart the kernel to use updated packages.


restart the kernel and run the imports

RUN THIS EACH TIME

# 0.5 Imports: run every time

In [2]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

DONT NEED TO RUN THIS EACH TIME

In [3]:
!nvidia-smi

Thu Feb 17 13:05:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

note the memory usage (0MiB / 32510MiB) for the Tesla V100.
You can re-rerun the above cell to see what memory your code/models are using during training.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are four released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the the current working directory at `/models/<model_name>`.

RUN THIS

In [4]:
model_name = "355M" # largest model we can fine-tune

run once to download the file

don't need to do this again:

In [5]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 450Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.71Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 1.67Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:22, 62.2Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.34Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.22Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 5.47Mit/s]                                                      


# 1. Generate Text From The Pretrained Model

If you want to generate text from the pretrained model pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`. (This is currently the only way to generate text from the 774M or 1558M models with this notebook.)

In [9]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

2022-02-17 13:06:50.062284: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-17 13:06:51.121202: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


## Sample from the model
The follow cell samples from gpt-2, using the provided prefix (seed) and other parameters. It starts the TF session and generates the samples.

Try changing the parameters below to change the output: 
- `prefix` is the prompt. This will be the starting string/seed for your generation. Use your own text. 
- `temperature` sets the variability/randomness of the output. Range 0.0-1.0
- `length` sets the lenght of output (in tokens). max is 1024.


In [10]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="My cat's name is " ,
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

My cat's name is _______. He's a great guy and loves me very much. He's been my friend for the last 10 years. He loves to cuddle, cuddle with me, and I love to cuddle with him. I've been a bit spoiled for affection over the years. I was never asked to do anything and I'm grateful for that. I didn't ask for a cat. I've been lucky to have a very loving, loving family. I've had good parents who always gave me
My cat's name is ____, and she's very cute and sweet." I really like cats, so I asked her to name me after her. The cat is named ____.

"My cat's name is ____, and she's very cute and sweet." I really like cats, so I asked her to name me after her. The cat is named ____. "My cat's name is ____, and she's very cute and sweet."

"My cat's name is ____
My cat's name is _______ and she's a young, beautiful, and very active little cat. She has a very special love of things that sparkle and sparkle and sparkle and sparkle. I've been very lucky to have her for several years now, and I think

## Activities
- try varying the prefix. 
  - what length of prefix works best with the given model? 
  - how does the choice of prefix change the format/form of the output.
- try varying the temperature.
- try loading the different sized models (124M, 355M, 774M, 1558M) and generate text without changing the other parameters. 
  - Do you notice any qualitative differences in the output? 

# 2. Fine-Tuning GPT-2

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [13]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-17 13:07:13.816590: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


## Upload a text file
For this, we will use a text file you provide to finetune (continue training) GPT-2. You can use any plain text (.txt) file. 

Simply drag and dropy our text file into the file browser at left. 

Once you have uploaded your file, update the file name in the cell below, then run it.

In [11]:
file_name = "cats.txt" # your file here

## Run the finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every `save_every` steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them. If your input text is smaller, training might proceed more quickly.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [14]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=100,            #depending on size of text this changes (maybe 1000?)
              restore_from='fresh', # change to 'latest' to resume
              run_name='run1',
              print_every=8,
              learning_rate=1e-5,
              #learning rate smaller, learns less per step
              sample_every=50,
              save_every=100
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1



2022-02-17 13:07:25.364251: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 105.64it/s]

dataset has 9182 tokens
Training...


[8 | 22.03] loss=2.47 avg=2.47
[16 | 37.88] loss=2.27 avg=2.37
[24 | 53.73] loss=1.97 avg=2.23
[32 | 69.58] loss=2.12 avg=2.20
[40 | 85.45] loss=1.88 avg=2.14
[48 | 101.30] loss=1.79 avg=2.08
======== SAMPLE 1 ========
 stimulation, the effect of the test object on heart rate decreases with increasing time spent in a closed position. In addition, the response to novelty may be increased with increasing time spent in a closed position. These results suggest that an object that is either in the open or closed position can generate an effect similar to that of an active computer mouse.<|endoftext|>If you're as bored as me in front of the TV as I am and only have one thing to do — take a stroll down memory lane as I make the rounds of this year's Anime Expo. And if only it wasn't the nostalgia that's making me so excited! After all, the year is 2000, so it might have been a good time for me to get a little nostalgic. I mean, I had all my favorite anime growing up, right?

Well, you know wh

## Notes on finetuning

Keep an eye on the loss, and how quickly it is dropping. A too-rapid drop in loss could be a sign of overfitting, and a learning rate (lr) that is too high. 

After the model is trained, you can download the checkpoint folder to save your work. Training checkpoints are saved to `checkpoint/run1` (or whatever you chose for the run name above).

You can compress it to a rar file and download that. Ask the instructor how.

You're done! Feel free to go to the Generate Text From The Trained Model section to generate text based on your retrained model.

## Finetune some more

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [ ]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

To fine-tune some more, run the following. Be sure to increase the number of steps (if it was `500` before, change to `1000` to train for 500 more. the number is cumulative).

In [ ]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=20,
              restore_from='latest', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=100,
              save_every=100
              )

# 3. Generate Text From The Finetuned Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [15]:
gpt2.generate(sess, run_name='run1') # no prefix, unconditional generation, tells you what it has learned without prefix

In short, Beale is a data journalist, so his research should be more in-depth than what you'll find in his articles. He is a big fan of statistics and therefore he will be fascinated by the physical properties of things. Beale has been a student of physiology for a while now, and he has been encouraged by the work of colleagues such as Dr. Rebecca Bardwell at the University of Edinburgh to pursue a career in this area. Beale knows his subjects well, and he is interested in hearing their perspectives on the world. Beale will be particularly interested in hearing about the challenges they face, and how they are working to overcome them. Beale is curious about the social and cultural changes taking place in these areas, and he will be interested to know how people are adapting to change. Beale is a big reader, and he will be looking forward to reading something he has never before been able to read. Beale is highly motivated, eager to learn, and enthusiastic about his field of work. He is

In [16]:
gpt2.generate(sess, run_name='run1', prefix="Garfield")

Garfield" in 2007, and he was just as grumpy that year. He was a big "Fool!" in the movie, but he was more of a softie in real life. He loved playing with his cat toys and going for a walk. He was a total goofball with friends and family. He was very kind and gentle. He loved spending time with animals, especially cats, and was a big fan of nature programs. He loved books and the outdoors, and was a big fan of movies and TV shows. He enjoyed being with animals, and he was a good provider. He was spayed and neutered.

Gerald
Gerald is a 5-year old boy from Iowa who came to us with his foster mom because he was so aggressive toward other cats in his home. He was so hyper and would jump on your lap to play. He was spayed and well on his way to healthy weight. He has his playful moments and he is very affectionate but he needs his space! Gerald loves to play with other cats, but he is more of a quiet guy. He likes to be under a blanket, he likes being close to you, but he is not the guy wh

## Notes
If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [23]:
gpt2.generate(sess,
              length=400,
              temperature=0.8,
              prefix="Columbus",
              nsamples=5,
              batch_size=5
              )

Columbus Day Parade

6pm to?

Free Parking

Celebrate Columbus Day at the Columbus Day Parade! The parade will begin and end at the corner of East Towne and South Main Streets. You can take in the scenery from a distance or sit back and enjoy the music and entertainment of the weekend's top talent. You can even stop by the Downtown Farmers' Market to pick up some of your own fresh produce!

Saturday, June 1
Sunday, June 2
Farm to Table: Bitter Springs
10 am to 6 pm
In this community-sustaining program, participants help care for and feed desert tortoises at the Bitter Springs Marine Mammal Center. Volunteers cook for the animals and help them with daily maintenance tasks. Volunteers must be able to handle the work and be pet-friendly. Volunteers must be in good health.

Picnic with a Cause: Puget Sound
10 am to 5 pm
Come on out for a fun and family-friendly day out with your family! Kids can play a game or two, while parents can enjoy picnic tables, a fire ring, beer, and a good book.


In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

Download the file by hand in the browser at left.

## Load a Trained Model Checkpoint

Uploaded your saved checkpoint and unzip it.

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

This will reset or start the tensorflow session as necessary.

In [ ]:
tf.compat.v1.reset_default_graph()

if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

gpt2.load_gpt2(sess, run_name='run1')

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# License

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Reference
- Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
- Original repo: [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by [Max Woolf](http://minimaxir.com). 
- Original [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) from Max.